In [1]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [2]:
# Replace 'folder_path' with the actual path to your folder
folder_path = 'F:\FYP\BRATS_dataset\ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData'

# Get a list of all subfolder names in the folder
subfolder_names = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

# Print the list of subfolder names
print(subfolder_names)

['BraTS-GLI-00000-000', 'BraTS-GLI-00002-000', 'BraTS-GLI-00003-000', 'BraTS-GLI-00005-000', 'BraTS-GLI-00006-000', 'BraTS-GLI-00008-000', 'BraTS-GLI-00008-001', 'BraTS-GLI-00009-000', 'BraTS-GLI-00009-001', 'BraTS-GLI-00011-000', 'BraTS-GLI-00012-000', 'BraTS-GLI-00014-000', 'BraTS-GLI-00014-001', 'BraTS-GLI-00016-000', 'BraTS-GLI-00016-001', 'BraTS-GLI-00017-000', 'BraTS-GLI-00017-001', 'BraTS-GLI-00018-000', 'BraTS-GLI-00019-000', 'BraTS-GLI-00020-000', 'BraTS-GLI-00020-001', 'BraTS-GLI-00021-000', 'BraTS-GLI-00021-001', 'BraTS-GLI-00022-000', 'BraTS-GLI-00022-001', 'BraTS-GLI-00024-000', 'BraTS-GLI-00024-001', 'BraTS-GLI-00025-000', 'BraTS-GLI-00026-000', 'BraTS-GLI-00028-000', 'BraTS-GLI-00030-000', 'BraTS-GLI-00031-000', 'BraTS-GLI-00031-001', 'BraTS-GLI-00032-000', 'BraTS-GLI-00032-001', 'BraTS-GLI-00033-000', 'BraTS-GLI-00035-000', 'BraTS-GLI-00036-000', 'BraTS-GLI-00036-001', 'BraTS-GLI-00043-000', 'BraTS-GLI-00044-000', 'BraTS-GLI-00045-000', 'BraTS-GLI-00045-001', 'BraTS-GLI

# Functions to add Motion Artifacts

In [3]:
def fourier_transform(image):
    if len(image.shape) == 3:
        image = np.mean(image, axis=-1)

    # Perform 2D Fourier Transform
    fft_result = fft2(image)

    # Shift zero frequency components to the center
    fft_result_shifted = fftshift(fft_result)

    # Calculate the magnitude spectrum (log scale for better visualization)
    magnitude_spectrum = np.log(np.abs(fft_result_shifted) + 1)

    return magnitude_spectrum,fft_result_shifted

def inverse_fourier(fft_result_shifted):
    # Inverse shift to prepare for inverse Fourier transform
    fft_result_unshifted_masked = ifftshift(fft_result_shifted)
    # Inverse Fourier Transform
    original_filtered = np.abs(ifft2(fft_result_unshifted_masked))
    return original_filtered


def add_rotation(angle, image):
    # Define the rotation angle (in degrees)

    # Get the center of the image
    height, width = image.shape[:2]
    center = (width / 2, height / 2)

    # Define the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    return rotated_image
        

# Generate Motion Artifacts

In [6]:
import cv2
from sklearn.preprocessing import LabelEncoder

for index,folder in enumerate(subfolder_names):
    parent_path = f'F:\FYP\BRATS_dataset\ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData\{folder}'
    if (index<25):
        for slice_number in range(75,95):

            test_image_seg= nib.load(f"{parent_path}/{folder}-seg.nii.gz").get_fdata()
            test_image_seg = test_image_seg.astype(np.int8)
            # test_image_seg= scaler.fit_transform(test_image_seg.reshape(-1, test_image_seg.shape[-1])).reshape(test_image_seg.shape) #convert to a range of [0,1]
            test_image_seg = test_image_seg[:,:,slice_number] #86 is the slice taken
            
            # labelencoder = LabelEncoder()
            # train_masks_reshaped = test_image_seg.reshape(-1,1)
            # train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
            # test_image_seg = train_masks_reshaped_encoded.reshape(240,240)
            

            test_image_t1c=nib.load(f"{parent_path}/{folder}-t1c.nii.gz").get_fdata()
            test_image_t1c=scaler.fit_transform(test_image_t1c.reshape(-1, test_image_t1c.shape[-1])).reshape(test_image_t1c.shape)
            test_image_t1c = test_image_t1c[:,:,slice_number]


            test_image_t1n=nib.load(f"{parent_path}/{folder}-t1n.nii.gz").get_fdata()
            test_image_t1n=scaler.fit_transform(test_image_t1n.reshape(-1, test_image_t1n.shape[-1])).reshape(test_image_t1n.shape)
            test_image_t1n = test_image_t1n[:,:,slice_number]

            test_image_t2f=nib.load(f"{parent_path}/{folder}-t2f.nii.gz").get_fdata()
            test_image_t2f=scaler.fit_transform(test_image_t2f.reshape(-1, test_image_t2f.shape[-1])).reshape(test_image_t2f.shape)
            test_image_t2f = test_image_t2f[:,:,slice_number]

            test_image_t2w=nib.load(f"{parent_path}/{folder}-t2w.nii.gz").get_fdata()
            test_image_t2w=scaler.fit_transform(test_image_t2w.reshape(-1, test_image_t2w.shape[-1])).reshape(test_image_t2w.shape)
            test_image_t2w = test_image_t2w[:,:,slice_number]

            #Add rotation here
            chosen_image = test_image_t2f
            magnitude_spectrum_2,fft_result_shifted_2 = fourier_transform(chosen_image)

            number_of_lines = 35 # Define the number of lines to be Sampled from 
            width = 1
            angle = 7
            # line_indexes = np.random.randint(0, chosen_image.shape[1]-2, number_of_lines).astype(int)
            line_indexes = np.linspace(0, 254, number_of_lines).astype(int)
            motion_corrupted = fft_result_shifted_2

            rotated_image = add_rotation(angle,test_image_t2f)
            rotated_magnitude_spectrum,rotated_fft_result_shifted = fourier_transform(rotated_image)

            for i in line_indexes:
                extracted = rotated_fft_result_shifted[i:i+width,:]
                motion_corrupted[i:i+width,:] = extracted
            
            inverse_fourier_corrupted = inverse_fourier(motion_corrupted)

            # plt.subplot(231)
            # plt.imshow(test_image_seg, cmap='jet')
            # plt.title('Label')
            # plt.subplot(232)
            # plt.imshow(chosen_image, cmap='gray')
            # plt.title('Image t2f')
            # plt.subplot(233)
            # plt.imshow(inverse_fourier_corrupted, cmap='gray')
            # plt.title('Corrupted')
            # plt.show()

            # os.mkdir(f"F:\FYP\FYP\motion_blur_repo\\brats_scans\scan_{path}")

            cv.imwrite(f'F:\FYP\FYP\BRATS_training_data\Original\{folder}_slice_{slice_number}_t2f_simulation_1.png', chosen_image*255)
            cv.imwrite(f'F:\FYP\FYP\BRATS_training_data\Corrupted\{folder}_slice_{slice_number}_corrupted_simulation_1.png', inverse_fourier_corrupted*255)
            plt.imsave(f'F:\FYP\FYP\BRATS_training_data\Label\{folder}_slice_{slice_number}_Label_simulation_1.png',test_image_seg,cmap="jet")
            # cv.imwrite(f'F:\FYP\FYP\BRATS_training_data\Label\{folder}_slice_{slice_number}_Label_simulation_1.png', test_image_seg*255)

KeyboardInterrupt: 